In [7]:
import pandas as pd

# Import database from excel sheet. It is organized in columns [variable | unit | symbol | value]
# You can set index col (you can take symbol for cleaner formulas)
df = pd.read_excel("../../database/ml_projekty.xlsx", sheet_name="Base", index_col="index", skiprows=1)

display(df.head(10))
# to get value us df.at[<index>, "value"]



,alpha,cz_inter,cx_inter,cz,cx,cx',c'x_p,a_i [deg],a_p [deg],cz_h,cx_h,cx_szkodliwe,C_x samolot,C_z samolotu,K,E,V,w,gamma
index,,,,,,,,,,,,,,,,,,,
1,-18.514286,-0.847525,0.012299,-0.847458,0.012304,0.011883,0.048312,-2.402232,-20.916518,-0.200086,0.015438,0.040989,0.097975,-0.876871,-8.949933,-70.238479,NaN,NaN,NaN
2,-17.142857,-0.886094,0.013129,-0.887006,0.013149,0.012733,0.052466,-2.511554,-19.654412,-0.204033,0.015575,0.041285,0.102757,-0.916019,-8.914403,-72.792910,NaN,NaN,NaN
3,-16.571429,-0.940679,0.014393,-0.937853,0.014526,0.014028,0.058686,-2.666269,-19.237697,-0.209619,0.015774,0.041703,0.109892,-0.971423,-8.839820,-75.909309,NaN,NaN,NaN
4,-16.114286,-0.992422,0.016608,-0.994350,0.016507,0.016272,0.065871,-2.812931,-18.927217,-0.214914,0.015967,0.042100,0.118034,-1.023943,-8.674987,-77.057228,NaN,NaN,NaN
5,-15.085714,-1.072474,0.020162,-1.073446,0.020234,0.019870,0.077633,-3.039831,-18.125545,-0.223106,0.016275,0.042713,0.131325,-1.105196,-8.415753,-78.275414,NaN,NaN,NaN
6,-13.485714,-1.065828,0.019625,-1.062147,0.019626,0.019330,0.076391,-3.020994,-16.506708,-0.222426,0.016249,0.042662,0.129937,-1.098451,-8.453692,-78.500668,NaN,NaN,NaN
7,-12.000000,-0.975958,0.016001,-0.983051,0.016069,0.015656,0.063654,-2.766264,-14.766264,-0.213229,0.015905,0.041973,0.115517,-1.007231,-8.719336,-76.576584,NaN,NaN,NaN
8,-10.742857,-0.866182,0.012716,-0.858757,0.012528,0.012309,0.050318,-2.455115,-13.197972,-0.201995,0.015504,0.041132,0.100285,-0.895808,-8.932629,-71.478182,NaN,NaN,NaN
9,-9.371429,-0.729562,0.010536,-0.734463,0.010711,0.010054,0.037294,-2.067878,-11.439306,-0.188014,0.015036,0.040085,0.085155,-0.757137,-8.891272,-59.855290,NaN,NaN,NaN
